<a href="https://colab.research.google.com/github/ArmstrongVo/VoTrongLuan.github.io/blob/main/TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# import libraries
import keras
import cv2
import tensorflow as tf
from keras.models import Sequential
from glob import glob
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from keras.layers import LeakyReLU

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# access to data
test='/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/TEST'
train='/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/TRAIN'

In [10]:
# để hệ thống đạt được độ chính xác cao nhất
# tiến hành pre-processing sử dụng kỹ thuật data agumentation
def equalize_histogram(image):   
    # chuẩn hóa độ sáng trong khoảng [0-1] : tối ưu độ sáng và chi tiết trong ảnh
    image = image.astype(np.float32) / 255.0
   
    # kiểm tra nếu hình ảnh có nhiều hơn 2 chiều thì chuyển đổi sang ảnh xám
    if len(image.shape) > 2:  
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     # Kiểm tra kiểu dữ liệu ảnh
    if image.dtype != np.uint8:
        image = image.astype(np.uint8)
    # Cân bằng histogram 
    equalized_image = cv2.equalizeHist(image)
    equalized_image = np.expand_dims(equalized_image, axis=-1)
    
    return equalized_image

# thiết lập các thông số cho data augmentation
# cân bằng histogram
# xoay ảnh
# lật chiều ngang
# lật dọc
# phóng to hoặc thu nhỏ
train_augen = ImageDataGenerator(preprocessing_function=equalize_histogram,
                                   rotation_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2)

test_augen = ImageDataGenerator(preprocessing_function=equalize_histogram,
                                   rotation_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2)
# data augumentation
train_set = train_augen.flow_from_directory(train,
                                                 target_size=(64, 64),
                                                 batch_size=16,
                                                 class_mode='categorical')
test_set = test_augen.flow_from_directory(test,
                                            target_size=(64, 64),
                                            batch_size=8,
                                            class_mode='categorical')


Found 5478 images belonging to 3 classes.
Found 370 images belonging to 3 classes.


In [11]:
# cấu trúc CNN
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(64, 64,1)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(8, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Activation("relu"))
model.add(Dense(1024))
model.add(Dropout(0.4))
model.add(Dense(3))
model.add(Activation("softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 16)         0

In [ ]:
# compile mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training_model = model.fit(train_set,
                    steps_per_epoch=5478//16,
                    epochs=100,
                    validation_data=test_set,
                    validation_steps=370//8)

history=model.fit(train_set,epochs=30,batch_size=128,verbose=1)                   